# Assignment-4

`assignment4.csv`  is the data for you to do analysis on.  It is the data to predict cars' prices.

1. This data set is the same as Assignment 3
 - The complete documentation on the dataset can be found here for your reference: https://archive.ics.uci.edu/ml/datasets/Automobile
2. Before doing prediction, you will need to preprocess the data as what you did in Assignment 3
 - Delete rows with missing data points (denoted with "?")
 - You should convert the data types of some features to the appropriate data types (i.e. float, integer, etc). Refer to the documentation
 - You should be able to distinguish numerical from categorical features. Refer to documentation
 - target = \['price'\]
 - When doing "train_test_split", use random_state = 20190306
3. The task is to use LASSO regression combined with $k$-fold to find a best feature subset which gives us the smallest square root of mean square error corresponding validation set. Use k = 10. Then compute the  square root of mean square error of test set.
4. lambda_list = np.array(\[0.001,0.01,0.1,1,10,100,1000,10000\])
5. Finally, use Elastic Net with $k$-fold to find the best feature subset and calculate the RMSE. Use k = 10, l1_ratio = 1/3 and the lambda_list defined in 4. 

Note: Do not shut down the kernel when you run the codes.

Hint: Try to think about the difference between Ridge and LASSO (google sklearn LASSO and sklearn Ridge) <br>
Hint 2: sklearn has a package for Elastic Net. The usage is straightforward if you understand how Elastic Net works.

In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import train_test_split, KFold
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import mean_squared_error


In [2]:
pd.read_csv('assignment4.csv', index_col = 0)
data = pd.read_csv('assignment4.csv').drop(['Unnamed: 0'], axis = 1)
data = data[~(data == '?').any(axis=1)]

In [3]:
data[['normalized-losses','bore','stroke','horsepower','peak-rpm','price']].astype('float')
features_numerical = ['normalized-losses','wheel-base','length','width','height','curb-weight','engine-size','bore','stroke','compression-ratio','horsepower','peak-rpm','city-mpg','highway-mpg']
target = ['price']
features_categorical = list(set(data.columns) - set(features_numerical) - set(target))
features = features_categorical + features_numerical
sales = data[features_numerical + features_categorical + target]

In [4]:
def onehot_encoder(df, feature):
    result = pd.DataFrame()
    if feature in df.columns:
        # The following line is important, refer to Assignment 2
        result = pd.get_dummies(df, columns=[feature])
        return result
    else:
        return print("Please select a feature in this df!")

In [5]:
def kfold_Lasso(data, target, n, lamb):
    # We need a vector to record mse from k-fold
    MSE = np.array([]) 
    
    # KFold is a build-in function in Scikit-learn
    #    it can help us cut data into n pieces 
    #    (compare with simple cross validation)
    kf = KFold(n_splits = n, random_state = 20190306, shuffle=True)
    
    for train_index, validation_index in kf.split(data):

        # obtain the train and validation part
        train, valid = data.loc[train_index,:], data.loc[validation_index,:]
        
        # extract X and Y to be fit in a Ridge regression model
        X_train = train.drop(target, axis = 1)
        Y_train = train[target]        
        X_valid = valid.drop(target, axis = 1)
        Y_valid = valid[target]

        # build Ridge regression model
        model = linear_model.Lasso(alpha = lamb)
        
        # fit model using training data
        model.fit(X_train,Y_train)
        
        # predict using validation data
        Y_valid_fit = model.predict(X_valid)
        
        # Calculate MSE
        MSE_temp = mean_squared_error(Y_valid_fit, Y_valid)
        
        # Add MSE to the list
        MSE = np.append(MSE, MSE_temp)
        
    return MSE.mean()

In [6]:
lambda_list = np.array([0.001,0.01,0.1,1,10,100,1000,10000])
MSE_Lasso = np.array([])

In [7]:
for i in features_categorical:
    sales = onehot_encoder(sales, i)    

# Split
train_valid, test = train_test_split(sales, test_size = 0.2, random_state = 20190306)
train_valid = train_valid.reset_index(drop = True)
test = test.reset_index(drop = True)

for l in lambda_list:
    MSE_Lasso = np.append(MSE_Lasso, kfold_Lasso(train_valid,'price',10, l))

MSE_Lasso

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.433e+07, tolerance: 3.449e+05
  model = cd_fast.enet_coordinate_descent(
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.764e+07, tolerance: 4.348e+05
  model = cd_fast.enet_coordinate_descent(
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.566e+07, toleranc

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.608e+07, tolerance: 4.355e+05
  model = cd_fast.enet_coordinate_descent(
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.035e+06, tolerance: 4.083e+05
  model = cd_fast.enet_coordinate_descent(


array([5392054.38983597, 5355673.9127132 , 5011147.04539408,
       4860544.63167275, 6283318.30561949, 6767653.71065752,
       7988211.4764678 , 7188923.47159935])

In [8]:
MSE_Lasso

array([5392054.38983597, 5355673.9127132 , 5011147.04539408,
       4860544.63167275, 6283318.30561949, 6767653.71065752,
       7988211.4764678 , 7188923.47159935])

In [9]:
lambda_selected = lambda_list[MSE_Lasso.argmin()]
lambda_selected

1.0

In [10]:
# we build linear regression model
model_Lasso = linear_model.Lasso(alpha=lambda_selected)

# features traget split
X_train_valid = train_valid.drop(target, axis = 1)
Y_train_valid = train_valid[target]

# fit the model
model_Lasso.fit(X_train_valid, Y_train_valid)

# use the model
X_test = test.drop(target, axis = 1)
Y_test = test[target]

Y_test_fit = model_Lasso.predict(X_test)
mean_squared_error(Y_test_fit, Y_test)

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.661e+07, tolerance: 4.565e+05
  model = cd_fast.enet_coordinate_descent(


2848659.9596595084

In [11]:
np.sqrt(mean_squared_error(Y_test_fit, Y_test))

1687.7973692536402

In [12]:
feature_list = np.delete(sales.columns.values, np.where(sales.columns.values == target))[model_Lasso.coef_>0]
i = 0
while i < len(feature_list):
    loc = feature_list[i].find('_')
    if loc > -1:
        feature_name = feature_list[i][0:loc]
        feature_list = np.delete(feature_list, i)
        i -= i 
        if feature_name not in feature_list:
            feature_list = np.append(feature_list, feature_name)
    i += 1
print("Number of features selected: " + str(len(feature_list)))
print(feature_list)

Number of features selected: 14
['wheel-base' 'width' 'curb-weight' 'engine-size' 'highway-mpg'
 'aspiration' 'body-style' 'num-of-cylinders' 'num-of-doors' 'symboling'
 'make' 'fuel-system' 'engine-type' 'drive-wheels']


In [13]:
def kfold_ElasticNet(data, target, n, lamb):
    # We need a vector to record mse from k-fold
    MSE = np.array([]) 
    
    # KFold is a build-in function in Scikit-learn
    #    it can help us cut data into n pieces 
    #    (compare with simple cross validation)
    kf = KFold(n_splits = n, random_state = 20190306, shuffle=True)
    
    for train_index, validation_index in kf.split(data):

        # obtain the train and validation part
        train, valid = data.loc[train_index,:], data.loc[validation_index,:]
        
        # extract X and Y to be fit in a Ridge regression model
        X_train = train.drop(target, axis = 1)
        Y_train = train[target]        
        X_valid = valid.drop(target, axis = 1)
        Y_valid = valid[target]

        # build Ridge regression model
        model = linear_model.ElasticNet(alpha = lamb, l1_ratio = 1/3, random_state = 20190306)
        
        # fit model using training data
        model.fit(X_train,Y_train)
        
        # predict using validation data
        Y_valid_fit = model.predict(X_valid)
        
        # Calculate MSE
        MSE_temp = mean_squared_error(Y_valid_fit, Y_valid)
        
        # Add MSE to the list
        MSE = np.append(MSE, MSE_temp)
        
    return MSE.mean()

In [14]:
MSE_ElasticNet = np.array([]) 

# Split
train_valid, test = train_test_split(sales, test_size = 0.2, random_state = 20190306)
train_valid = train_valid.reset_index(drop = True)
test = test.reset_index(drop = True)

for lamb in lambda_list:
    MSE_ElasticNet = np.append(MSE_ElasticNet, kfold_ElasticNet(train_valid,'price',10, lamb))

MSE_ElasticNet

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.173e+07, tolerance: 3.449e+05
  model = cd_fast.enet_coordinate_descent(
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.592e+07, tolerance: 4.348e+05
  model = cd_fast.enet_coordinate_descent(
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.319e+07, toleranc

array([4132093.41426942, 4952096.91955969, 6116316.87378376,
       6846500.27294865, 7573867.85945378, 7509473.4880744 ,
       7247848.70976864, 7200705.72873158])

In [15]:
lambda_selected = lambda_list[MSE_Lasso.argmin()]
lambda_selected

1.0

In [16]:
# we build linear regression model
model_ElasticNet = linear_model.ElasticNet(alpha=lambda_selected)

# features traget split
X_train_valid = train_valid.drop(target, axis = 1)
Y_train_valid = train_valid[target]

# fit the model
model_ElasticNet.fit(X_train_valid, Y_train_valid)

# use the model
X_test = test.drop(target, axis = 1)
Y_test = test[target]

Y_test_fit = model_ElasticNet.predict(X_test)
mean_squared_error(Y_test_fit, Y_test)

7973032.673162295

In [17]:
np.sqrt(mean_squared_error(Y_test_fit, Y_test))

2823.6559055880543

In [18]:
feature_list = np.delete(sales.columns.values, np.where(sales.columns.values == target))[model_ElasticNet.coef_>0]
i = 0
while i < len(feature_list):
    loc = feature_list[i].find('_')
    if loc > -1:
        feature_name = feature_list[i][0:loc]
        feature_list = np.delete(feature_list, i)
        i -= i 
        if feature_name not in feature_list:
            feature_list = np.append(feature_list, feature_name)
    i += 1
print("Number of features selected: " + str(len(feature_list)))
print(feature_list)

Number of features selected: 18
['wheel-base' 'width' 'curb-weight' 'engine-size' 'compression-ratio'
 'horsepower' 'peak-rpm' 'city-mpg' 'aspiration' 'body-style'
 'num-of-cylinders' 'fuel-type' 'num-of-doors' 'symboling' 'make'
 'fuel-system' 'engine-type' 'drive-wheels']
